In [1]:
%%capture
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [2]:
%%capture
# Import the relevant packages

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from re import search
# import sklearn as sk
# from sklearn.preprocessing import OneHotEncoder
import torch 

In [3]:
# Import the relevant data sets

train = pd.read_csv('../input/us-patent-phrase-to-phrase-matching/train.csv')
train.set_index('id') # Set the ID column as the index
print(train.shape)
test = pd.read_csv('../input/us-patent-phrase-to-phrase-matching/test.csv',index_col = 'id')
print(test.shape)

codes = pd.read_csv('../input/cpc-titles/title_info.csv')
print(codes.head())

codes_copy = codes.copy() # Create a copy of the cpc_titles file
codes = codes.rename(columns = {"code" : "context"}) # Rename the "code" column to be more in line with our requirements
display(codes)

(36473, 5)
(36, 3)
       code                                              title section  class  \
0         A                                  HUMAN NECESSITIES       A    NaN   
1       A01  AGRICULTURE; FORESTRY; ANIMAL HUSBANDRY; HUNTI...       A    1.0   
2      A01B  SOIL WORKING IN AGRICULTURE OR FORESTRY; PARTS...       A    1.0   
3  A01B1/00  Hand tools (edge trimmers for lawns A01G3/06  ...       A    1.0   
4  A01B1/02  Spades; Shovels {(hand-operated dredgers E02F3...       A    1.0   

  subclass  group  main_group  
0      NaN    NaN         NaN  
1      NaN    NaN         NaN  
2        B    NaN         NaN  
3        B    1.0         0.0  
4        B    1.0         2.0  


,context,title,section,class,subclass,group,main_group
0,A,HUMAN NECESSITIES,A,NaN,NaN,NaN,NaN
1,A01,AGRICULTURE; FORESTRY; ANIMAL HUSBANDRY; HUNTI...,A,1.0,NaN,NaN,NaN
2,A01B,SOIL WORKING IN AGRICULTURE OR FORESTRY; PARTS...,A,1.0,B,NaN,NaN
3,A01B1/00,Hand tools (edge trimmers for lawns A01G3/06 ...,A,1.0,B,1.0,0.0
4,A01B1/02,Spades; Shovels {(hand-operated dredgers E02F3...,A,1.0,B,1.0,2.0
...,...,...,...,...,...,...,...
260471,Y10T483/1864,including tool pot or adapter,Y,10.0,T,483.0,1864.0
260472,Y10T483/1873,Indexing matrix,Y,10.0,T,483.0,1873.0
260473,Y10T483/1882,Rotary disc,Y,10.0,T,483.0,1882.0
260474,Y10T483/1891,Chain or belt,Y,10.0,T,483.0,1891.0


In [4]:
context = train['context'].unique() # Create a numpy array to store the distinct context identifiers
print(context.shape)
# print(context)

anchor = train['anchor'].unique() # Create a numpy array to store the distinct anchors
print(type(anchor))
print("Anchor unique values")
print(anchor.shape)

target = train['target'].unique() # Create a numpy array to store the distinct targets
print(target.shape)

(106,)
<class 'numpy.ndarray'>
Anchor unique values
(733,)
(29340,)


In [5]:
train_merged = pd.merge(train, codes[["context","title"]], on="context", how="left") # Merge the training set and cpc_titles for further analysis
train_merged.set_index('id') # Set the ID column as the index
print(train_merged.index)
print(train_merged.head())
print(train_merged.shape)
#print(type(train_merged))

anchor_matrix = np.asmatrix(anchor) # Convert the anchor array into a n-D matrix; where n = number of unique anchors 
print("matrix shape")
print(anchor_matrix.shape)
print("matrix")
print(anchor_matrix)

Int64Index([    0,     1,     2,     3,     4,     5,     6,     7,     8,
                9,
            ...
            36463, 36464, 36465, 36466, 36467, 36468, 36469, 36470, 36471,
            36472],
           dtype='int64', length=36473)
                 id     anchor                  target context  score  \
0  37d61fd2272659b1  abatement  abatement of pollution     A47   0.50   
1  7b9652b17b68b7a4  abatement          act of abating     A47   0.75   
2  36d72442aefd8232  abatement         active catalyst     A47   0.25   
3  5296b0c19e1ce60e  abatement     eliminating process     A47   0.50   
4  54c1e3b9184cb5b6  abatement           forest region     A47   0.00   

                                               title  
0  FURNITURE; DOMESTIC ARTICLES OR APPLIANCES; CO...  
1  FURNITURE; DOMESTIC ARTICLES OR APPLIANCES; CO...  
2  FURNITURE; DOMESTIC ARTICLES OR APPLIANCES; CO...  
3  FURNITURE; DOMESTIC ARTICLES OR APPLIANCES; CO...  
4  FURNITURE; DOMESTIC ARTICLES OR APPLIA

In [6]:
%%capture

# The conversion to matrix and then to a dataframe ensures that we're not dealing with a 1-D matrix/vector
# A 1-D matrix/vector will be useless to us as we need the anchors in the columns for the one-hot encoding

In [7]:
df = pd.DataFrame(anchor_matrix) # Convert the matrix into a dataframe as this will make the following steps easier
print("matrix to dataframe shape")
print(df.shape)
print("matrix to dataframe structure")
print(df)
df.columns = df.iloc[0] # This ensures that the rows values are considered for column names; here we only have one row
print("after converting rows to column headers")
df = df.drop([0]) # We drop the row prior to the merge; this will create an empty dataframe
print(df)

matrix to dataframe shape
(1, 733)
matrix to dataframe structure
         0                  1                     2     3    \
0  abatement  abnormal position  absorbent properties  acan   

                  4                       5                6    7    \
0  accept information  achieve authentication  acid absorption  ack   

                      8                9    ...              723          724  \
0  acoustooptic modulator  acrylate groups  ...  wave generation  wearability   

            725           726          727        728             729  \
0  web transfer  wedge device  weldability  well pump  whose identity   

                      730            731           732  
0  wire grid polarization  wiring trough  wood article  

[1 rows x 733 columns]
after converting rows to column headers
Empty DataFrame
Columns: [abatement, abnormal position, absorbent properties, acan, accept information, achieve authentication, acid absorption, ack, acoustooptic modulator, acr

In [8]:
train_merged = pd.concat([train_merged,df], axis=1) # Concat the new columns to the training dataset
print(train_merged.shape)
print(train_merged.head(4))

(36473, 739)
                 id     anchor                  target context  score  \
0  37d61fd2272659b1  abatement  abatement of pollution     A47   0.50   
1  7b9652b17b68b7a4  abatement          act of abating     A47   0.75   
2  36d72442aefd8232  abatement         active catalyst     A47   0.25   
3  5296b0c19e1ce60e  abatement     eliminating process     A47   0.50   

                                               title abatement  \
0  FURNITURE; DOMESTIC ARTICLES OR APPLIANCES; CO...       NaN   
1  FURNITURE; DOMESTIC ARTICLES OR APPLIANCES; CO...       NaN   
2  FURNITURE; DOMESTIC ARTICLES OR APPLIANCES; CO...       NaN   
3  FURNITURE; DOMESTIC ARTICLES OR APPLIANCES; CO...       NaN   

  abnormal position absorbent properties acan  ... wave generation  \
0               NaN                  NaN  NaN  ...             NaN   
1               NaN                  NaN  NaN  ...             NaN   
2               NaN                  NaN  NaN  ...             NaN   
3         

In [9]:
temp = train_merged.copy() # Make a copy of the training set
temp = temp.drop(columns=["anchor","context","score","title"]) # Remove the irrelevant columns for experimentation
print(temp.head())

                 id                  target abatement abnormal position  \
0  37d61fd2272659b1  abatement of pollution       NaN               NaN   
1  7b9652b17b68b7a4          act of abating       NaN               NaN   
2  36d72442aefd8232         active catalyst       NaN               NaN   
3  5296b0c19e1ce60e     eliminating process       NaN               NaN   
4  54c1e3b9184cb5b6           forest region       NaN               NaN   

  absorbent properties acan accept information achieve authentication  \
0                  NaN  NaN                NaN                    NaN   
1                  NaN  NaN                NaN                    NaN   
2                  NaN  NaN                NaN                    NaN   
3                  NaN  NaN                NaN                    NaN   
4                  NaN  NaN                NaN                    NaN   

  acid absorption  ack  ... wave generation wearability web transfer  \
0             NaN  NaN  ...           

In [12]:
print("\nBefore conversion: ", type(anchor))
anchor_list = list(anchor)
print("\nAfter conversion: ", type(anchor_list))
#print(anchor_list)


Before conversion:  <class 'numpy.ndarray'>

After conversion:  <class 'list'>


In [25]:
for val in anchor: # Iterate through the 'anchor' array
        for i in temp['target']: # Iterate through the 'target' column in the 'temp'
            if search(i,val):
                    1.0
            else:
                    0.0

KeyboardInterrupt: 

In [24]:
# for r in temp[target]:
#     for c in temp[anchor]:
#         if search(r,c):
#                 1.0
#         else:
#                 0.0 

KeyError: "['abatement of pollution', 'act of abating', 'active catalyst', 'eliminating process', 'forest region', 'greenhouse gases', 'increased rate', 'measurement level', 'minimising sounds', 'mixing core materials', 'multi pollution abatement device', 'noise reduction', 'pollution abatement', 'pollution abatement incinerator', 'pollution certificate', 'rent abatement', 'sorbent material', 'source items pollution abatement technology', 'stone abutments', 'tax abatement', 'water bodies', 'emission abatement', 'prevention', 'prophylaxis', 'abatement apparatus', 'treatment', 'abating', 'central hub', 'control panel', 'deficiency', 'diminished chord', 'diminished image', 'diminishing', 'eliminating', 'eliminating emissions', 'eliminating pollutants', 'increasing', 'increasing order', 'mitigation', 'pollution control', 'processing unit', 'removing air pollutants', 'subsiding', 'treating emissions', 'noise abatement', 'abnormal position data', 'attitude', 'closed position shown', 'conditi